In [4]:
import numpy as np
import pandas as pd

In [32]:
test = pd.read_csv('include/test.csv')
train = pd.read_csv('include/train.csv')

In [48]:
def substrings_in_string(big_string, substrings):
    for substring in substrings:
        if str(big_string).find(substring) != -1:
            return substring
    return np.nan

In [53]:
train_reformated = train.drop(['PassengerId', 'Name'], axis=1)
cabin_list = ['A', 'B', 'C', 'D', 'E', 'F', 'T', 'G', 'Unknown']
train_reformated['Deck'] = train['Cabin'].map(lambda x: substrings_in_string(x, cabin_list))
train_reformated['Family_Size'] = train_reformated['SibSp'] + train_reformated['Parch']
train_reformated['Fare_Per_Person'] = train_reformated['Fare'] /(train_reformated['Family_Size'] + 1)
print(train_reformated)

     Survived  Pclass     Sex   Age  SibSp  Parch            Ticket      Fare  \
0           0       3    male  22.0      1      0         A/5 21171    7.2500   
1           1       1  female  38.0      1      0          PC 17599   71.2833   
2           1       3  female  26.0      0      0  STON/O2. 3101282    7.9250   
3           1       1  female  35.0      1      0            113803   53.1000   
4           0       3    male  35.0      0      0            373450    8.0500   
5           0       3    male   NaN      0      0            330877    8.4583   
6           0       1    male  54.0      0      0             17463   51.8625   
7           0       3    male   2.0      3      1            349909   21.0750   
8           1       3  female  27.0      0      2            347742   11.1333   
9           1       2  female  14.0      1      0            237736   30.0708   
10          1       3  female   4.0      1      1           PP 9549   16.7000   
11          1       1  femal